# Welcome to Airbnb Data Analysis's notebook
To run all the code, you can use the "run all" function. You can also follow along and run the code accordingly. Note that we assume this project to be running with ***Jupyter Notebook***.

## 1. Dependencies
We have recorded all the required dependencies in `requirements.txt`, to get all the dependencies installed, simply run the below's code.

In [1]:
%pip install -r requirements.txt --user